In [101]:
import pygsheets
from oauth2client.service_account import ServiceAccountCredentials
from pathlib import Path

In [76]:
import pandas as pd
pd.options.display.max_columns = 0

In [4]:

# Load the credentials from the JSON file (replace with the path to your file)
sf_path = r'C:\Users\kien.tn\burpee_dashboard\inductive-gift-355101-87f774556af8.json'
gc = pygsheets.authorize(service_account_file=sf_path)


In [10]:
def get_number_gsheet(service_file_path, spreadsheet_id, sheet_name):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(
        numerize=False, value_render='UNFORMATTED_VALUE', empty_value=None)
    df = df.replace(to_replace='', value=None)

    return df

# %%
# lay du lieu tu googhesheet ma khong co du lieu so


def get_df_from_gsheet(service_file_path, spreadsheet_id, sheet_name, start=None):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(start, numerize=False, empty_value=None)
    df = df.replace(to_replace='', value=None)
    df.drop_duplicates(inplace=True)

    return df

In [203]:
spreadsheet_id = '1Bmm9GGP3QE5k_T8eODN3Hrh8ICzMgAazYkw0gWNRZDs'
sheet_name = '9/2024'
start='A4'
gc = pygsheets.authorize(service_file=sf_path)
sh = gc.open_by_key(spreadsheet_id)
wks = sh.worksheet_by_title(sheet_name)
df = wks.get_as_df(start, numerize=False, value_render='UNFORMATTED_VALUE', empty_value=None)
df = df.replace(to_replace='', value=None)
df.drop_duplicates(inplace=True)

df2 = df.transpose()
df2.iloc[:,1] = df2.iloc[:,1].ffill()
df3 = df2.transpose()
# Combine the two lists
combined_list = [f"{a}|{b}" for a, b in zip(df3.iloc[1].to_list(), df3.iloc[2].to_list())]
df3.columns = combined_list

In [204]:
def extract_daily(df3, sheet_name):
    df_daily = df3.iloc[10:,:]
    df_daily2 = df_daily.melt(id_vars='None|None')
    df_daily2[['user', 'criteria']] =df_daily2['variable'].str.split('|', expand=True)
    df_daily2.rename(columns={'None|None':'report_date'}, inplace=True)
    df_daily2.drop(columns='variable', inplace=True)
    df_daily2.dropna(subset='value', inplace=True)
    df_daily3 = df_daily2.query("value!=0").pivot(columns='criteria', index=['user', 'report_date'], values='value').reset_index()
    df_daily3['report_date'] = df_daily3['report_date'].astype('str') + '/' + sheet_name
    df_daily3['report_date'] = pd.to_datetime(df_daily3['report_date'], format='%d/%m/%Y')
    # df_daily3.to_parquet(rf".\data\daily\{str(sheet_name).replace('/', '-')}.parquet", index=False )
    return df_daily3

In [205]:
def extract_target(df3, sheet_name):
    df_target = df3.iloc[3:10]
    df_target.rename(columns={"None|None":"item"}, inplace=True)
    df_target.query("item.isin(['Target', 'Daily'])", inplace=True)

    df_target2 = df_target.melt(id_vars='item')
    df_target2[['user', 'criteria']] =df_target2['variable'].str.split('|', expand=True)
    # df_target2.rename(columns={'item':'report_date'}, inplace=True)
    df_target2.drop(columns='variable', inplace=True)
    df_target2.dropna(subset='value', inplace=True)

    df_target2.drop_duplicates(subset=['item', 'user'], inplace=True)
    df_target3 = df_target2.query("value!=0").pivot(columns='item', index=['user'], values='value').reset_index()

    df_target3['report_month'] = '01/' + sheet_name
    df_target3['report_month'] = pd.to_datetime(df_target3['report_month'], format='%d/%m/%Y')
    # df_target3.to_parquet(rf".\data\target\{str(sheet_name).replace('/', '-')}.parquet", index=False )
    return df_target3

In [206]:
df_daily = extract_daily(df3, sheet_name)
df_target = extract_target(df3, sheet_name)

C:\Users\kien.tn\AppData\Local\Temp\ipykernel_20320\1832234757.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target.rename(columns={"None|None":"item"}, inplace=True)
C:\Users\kien.tn\AppData\Local\Temp\ipykernel_20320\1832234757.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target.query("item.isin(['Target', 'Daily'])", inplace=True)


In [216]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 390 entries, 0 to 12
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user               390 non-null    object        
 1   Daily              390 non-null    int32         
 2   Target             390 non-null    int32         
 3   report_month       390 non-null    datetime64[ns]
 4   report_date        390 non-null    datetime64[ns]
 5   report_day         390 non-null    int32         
 6   no_of_day          390 non-null    int64         
 7   daily_target_norm  390 non-null    float64       
 8   mtd_target_norm    390 non-null    float64       
dtypes: datetime64[ns](2), float64(2), int32(3), int64(1), object(1)
memory usage: 25.9+ KB


In [208]:
# Get start of month and number of days in the month
start_of_month = df_target.loc[0,'report_month']
num_days = start_of_month.days_in_month

# Create a date range for each day of the month
date_range = pd.date_range(start_of_month, periods=num_days, freq='D')
date_range
df_target['report_date'] = [date_range]*len(df_target)
df_target = df_target.explode('report_date')
df_target['report_day'] = df_target['report_date'].dt.day
df_target['no_of_day'] = num_days

In [209]:
df_target['Daily'] = df_target['Daily'].astype(int)


In [210]:
df_target['Target'] = df_target['Target'].astype(int)

In [213]:
df_target['daily_target_norm'] =  df_target['Target']/df_target["no_of_day"]

In [215]:
df_target['mtd_target_norm'] = df_target['daily_target_norm'] * df_target['report_day']

In [219]:
dta = df_target.merge(df_daily, how='left', on=['user','report_date'])

In [226]:
dta['Total'] = dta['Total'].fillna(0)

In [230]:
dta['mtd_actual'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Total'].cumsum()

In [234]:
dta['flg_daily']  = 0
flt = dta['Total'] >= dta['Daily']
dta.loc[flt, 'flg_daily'] = 1

In [236]:
dta['mtd_daily'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['flg_daily'].cumsum()

In [240]:
dta.to_parquet(rf".\data\dta_{str(sheet_name).replace('/', '-')}.parquet", index=False )

In [241]:
import plotly.express as px

In [385]:
dta_chart = dta.query("report_date=='2024-09-05'")

In [352]:
dta_chart.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 4 to 364
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user               13 non-null     object        
 1   Daily              13 non-null     int32         
 2   Target             13 non-null     int32         
 3   report_month       13 non-null     datetime64[ns]
 4   report_date        13 non-null     datetime64[ns]
 5   report_day         13 non-null     int32         
 6   no_of_day          13 non-null     int64         
 7   daily_target_norm  13 non-null     float64       
 8   mtd_target_norm    13 non-null     float64       
 9   Burpee             3 non-null      object        
 10  Core               2 non-null      object        
 11  Plank              1 non-null      object        
 12  Pushup             0 non-null      object        
 13  Squat              0 non-null      object        
 14  Total           

In [292]:
import plotly.graph_objects as go

# Parameters
total_amount = 1000  # Total target
current_value = 1100  # Current value

# Calculate percentage completion
percentage_complete = (current_value / total_amount) * 100

# Create the gauge chart
fig = go.Figure(go.Indicator(
    mode="gauge+number+delta",  # Shows the gauge, number, and delta (difference from total)
    value=current_value,  # Absolute value (current amount)
    number={'valueformat': ',.0f'},  # Display the absolute value without decimal points
    delta={'reference': total_amount, 'position': "top", 'increasing': {'color': "green"}},  # Show difference from total
    gauge={
        'axis': {'range': [0, total_amount]},  # Set the range from 0 to the total
        'bar': {'color': "blue"},  # Color of the gauge bar
        'steps': [
            {'range': [0, total_amount * 0.5], 'color': "#ff4545"},  # Colors for ranges
            {'range': [total_amount * 0.5, total_amount*0.75], 'color': "#fffc45"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': total_amount  # Mark the total with a threshold line
        }
    }
))

# Add annotation for percentage on a new line
fig.add_annotation(
    x=0.5, y=-0.1,  # Position the annotation
    text=f"({percentage_complete:.0f}%)",  # Display the percentage
    showarrow=False,
    font={'size': 20},
    xanchor="center",
    yanchor="bottom"
)

# Update layout to customize appearance
fig.update_layout(
    title={'text': "Sales Target Progress", 'x': 0.5},  # Title and center it
    height=400
)

# Display the chart
fig.show()


In [374]:
dta_chart.to_records()

rec.array([(  4, 'An',  40, 1000, '2024-09-01T00:00:00.000000000', '2024-09-05T00:00:00.000000000', 5, 30,  33.33333333, 166.66666667, nan, nan, nan, nan, nan,   0. ,  50. , 0, 1),
           ( 34, 'Hiệp',  56, 1400, '2024-09-01T00:00:00.000000000', '2024-09-05T00:00:00.000000000', 5, 30,  46.66666667, 233.33333333, nan, 120, nan, nan, nan,  60. , 115. , 1, 1),
           ( 64, 'Hoàng',  32,  800, '2024-09-01T00:00:00.000000000', '2024-09-05T00:00:00.000000000', 5, 30,  26.66666667, 133.33333333, nan, nan, nan, nan, nan,   0. ,   0. , 0, 0),
           ( 94, 'Kiên Chu',  32,  800, '2024-09-01T00:00:00.000000000', '2024-09-05T00:00:00.000000000', 5, 30,  26.66666667, 133.33333333, nan, nan, nan, nan, nan,   0. ,   0. , 0, 0),
           (124, 'Kiên Trần', 104, 2600, '2024-09-01T00:00:00.000000000', '2024-09-05T00:00:00.000000000', 5, 30,  86.66666667, 433.33333333, nan, nan, nan, nan, nan,   0. , 160. , 0, 1),
           (154, 'Ninh', 160, 4000, '2024-09-01T00:00:00.000000000', '2024-09

In [387]:
dta_chart.columns

Index(['user', 'Daily', 'Target', 'report_month', 'report_date', 'report_day',
       'no_of_day', 'daily_target_norm', 'mtd_target_norm', 'Burpee', 'Core',
       'Plank', 'Pushup', 'Squat', 'Total', 'mtd_actual', 'flg_daily',
       'mtd_daily'],
      dtype='object')

In [403]:
import plotly.graph_objects as go
import pandas as pd
import random


# Calculate percentage complete and differences
dta_chart['percentage_complete'] = (dta_chart['mtd_actual'] / dta_chart['Target']) * 100
dta_chart['difference'] = dta_chart['mtd_actual'] - dta_chart['Target']

# Function to generate random color
def random_color():
    return '#{:02x}{:02x}{:02x}'.format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Generate random colors for each store
dta_chart['color'] = [random_color() for _ in dta_chart['user']]

# Create the horizontal bar chart
fig = go.Figure()

# Add bars for each store
fig.add_trace(go.Bar(
    x=dta_chart['percentage_complete'],  # The percentage complete is shown in the bar
    y=dta_chart['user'],  # Store names
    orientation='h',  # Horizontal bar chart
    marker=dict(color=dta_chart['color']),  # Assign random colors to the bars
    text=[f"{row['user']} | {row['percentage_complete']:,.0f}%<br>Target: {row['Target']:,.0f}<br>Thực hiện: {row['mtd_actual']:,.0f}<br>Difference: {row['difference']:,.0f}"
          for _, row in dta_chart.iterrows()],  # Custom data labels
    textposition='outside',  # Position text at the right edge of the bar
    texttemplate='%{text}',  # Format to use custom text
    hoverinfo='none'  # Disable hover info to avoid duplication with text labels
    
))

# Customize layout
fig.update_layout(
    title="Thực hiện target tháng",
    xaxis_title="Percentage Complete",
    yaxis=dict(showticklabels=False),  # Hide the store names on the y-axis
    xaxis=dict(range=[0, dta_chart['percentage_complete'].max()*1.1]),  # Limit x-axis to 100% max
    height=len(dta_chart)*110,
    bargap=0.2,  # Space between bars
    showlegend=False
)

# Show the figure
fig.show()


C:\Users\kien.tn\AppData\Local\Temp\ipykernel_20320\2449174423.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kien.tn\AppData\Local\Temp\ipykernel_20320\2449174423.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kien.tn\AppData\Local\Temp\ipykernel_20320\2449174423.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [404]:
dta_chart.head()

,user,Daily,Target,report_month,report_date,report_day,no_of_day,daily_target_norm,mtd_target_norm,Burpee,Core,Plank,Pushup,Squat,Total,mtd_actual,flg_daily,mtd_daily,percentage_complete,difference,color
4,An,40,1000,2024-09-01,2024-09-05,5,30,33.333333,166.666667,NaN,NaN,NaN,NaN,NaN,0.0,50.0,0,1,5.000000,-950.0,#b3c58b
34,Hiệp,56,1400,2024-09-01,2024-09-05,5,30,46.666667,233.333333,NaN,120,NaN,NaN,NaN,60.0,115.0,1,1,8.214286,-1285.0,#b8c19c
64,Hoàng,32,800,2024-09-01,2024-09-05,5,30,26.666667,133.333333,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0.000000,-800.0,#1636a9
94,Kiên Chu,32,800,2024-09-01,2024-09-05,5,30,26.666667,133.333333,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0.000000,-800.0,#c4f83c
124,Kiên Trần,104,2600,2024-09-01,2024-09-05,5,30,86.666667,433.333333,NaN,NaN,NaN,NaN,NaN,0.0,160.0,0,1,6.153846,-2440.0,#79c540


In [408]:


# Calculate percentage complete and differences
dta_chart['percentage_complete'] = (dta_chart['mtd_actual'] / dta_chart['Target']) * 100
dta_chart['difference'] = dta_chart['mtd_actual'] - dta_chart['Target']

# Function to generate random color
def random_color():
    return '#{:02x}{:02x}{:02x}'.format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Generate random colors for each store
dta_chart['color'] = [random_color() for _ in dta_chart['user']]

# Create the horizontal bar chart
fig = go.Figure()

# Add bars for each store
fig.add_trace(go.Bar(
    x=dta_chart['mtd_daily'],  # The percentage complete is shown in the bar
    y=dta_chart['user'],  # Store names
    orientation='h',  # Horizontal bar chart
    marker=dict(color=dta_chart['color']),  # Assign random colors to the bars
    text=[f"{row['user']} | {row['mtd_daily']:,.0f} ngày"
          for _, row in dta_chart.iterrows()],  # Custom data labels
    textposition='outside',  # Position text at the right edge of the bar
    texttemplate='%{text}',  # Format to use custom text
    # hoverinfo='none'  # Disable hover info to avoid duplication with text labels
    
))

# Customize layout
fig.update_layout(
    title="Thực hiện target daily",
    # xaxis_title="Percentage Complete",
    yaxis=dict(showticklabels=False),  # Hide the store names on the y-axis
    # xaxis=dict(range=[0, 16]),  # Limit x-axis to 100% max
    height=len(dta_chart)*110,
    bargap=0.2,  # Space between bars
    showlegend=False
)

# Show the figure
fig.show()


C:\Users\kien.tn\AppData\Local\Temp\ipykernel_20320\2798183570.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kien.tn\AppData\Local\Temp\ipykernel_20320\2798183570.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kien.tn\AppData\Local\Temp\ipykernel_20320\2798183570.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or